Load label file

In [1]:
import json

path = './data/fish_classes.json'
file = open(path, "rb")
label_dict = json.load(file) 

images_labels store each fish and its corresponding label, and write it into train.txt

In [2]:
images_labels = {} 
with open("/home/xiaoer/chenchen/yolov5/data/fish_train.txt","r") as fin:
    lines = fin.readlines()
for line in lines:
    line = line.strip()
    if not line:
        continue
    temp = line.split(" ")
    if temp[0] not in images_labels:
        images_labels[temp[0]] = []
    label = label_dict[temp[1]+" "+temp[2]]
    images_labels[temp[0]] .append(str(label)+" "+ " ".join(temp[3:]))
print (len(images_labels))

5278


In [55]:
with open("/home/xiaoer/chenchen/yolov5/data/train.txt","w") as fout:
    for key,value in images_labels.items():
        fout.write(key+","+",".join(value)+"\n")

In [2]:
print (label_dict)

{'chaetodon trifascialis': 0, 'dascyllus reticulatus': 1, 'acanthurus nigrofuscus': 2, 'chaetodon lunulatus': 3, 'chaetodon speculum': 4, 'pempheris vanicolensis': 5, 'abudefduf vaigiensis': 6, 'chromis chrysura': 7, 'plectrogly-phidodon dickii': 8, 'hemigymnus melapterus': 9, 'myripristis kuntee': 10, 'zebrasoma scopas': 11, 'amphiprion clarkii': 12, 'neoglyphidodon nigroris': 13, 'dascyllus aruanus': 14}


How many corresponding pictures does each label have

In [3]:
import os 

train_dict = {}
for root, dirs, files in os.walk("/home/xiaoer/chenchen/yolov5/data/labels/train"):

    for f in files:
        with open (os.path.join(root, f)) as fin:
            lines = fin.readlines()
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                temp = line.split(" ")
                if temp[0] not in train_dict:
                    train_dict[temp[0]] = []
                if os.path.join(root, f) not in train_dict[temp[0]]:
                    train_dict[temp[0]].append(os.path.join(root, f))



In [4]:
for key,val  in train_dict.items():
    print (key+" "+str(len(val)))

7 298
1 2447
3 1051
5 777
12 385
6 188
0 341
8 634
14 788
9 241
10 271
4 202
2 302
13 147
11 129


In [5]:
print(train_dict["0"][-1])

/home/xiaoer/chenchen/yolov5/data/labels/train/1_10.txt


In [33]:
import os 

train_dict1 = {}
for root, dirs, files in os.walk("/home/xiaoer/chenchen/yolov5/data/labels/train"):

    for f in files:
        with open (os.path.join(root, f)) as fin:
            lines = fin.readlines()
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                temp = line.split(" ")
                if temp[0] not in train_dict1:
                    train_dict1[temp[0]] = []
                train_dict1[temp[0]].append(os.path.join(root, f))
print(len(train_dict1))

15


In [34]:
for key,val  in train_dict.items():
    print (key+" "+str(len(val)))

7 298
1 2447
3 1051
5 777
12 385
6 188
0 341
8 634
14 788
9 241
10 271
4 202
2 302
13 147
11 129


In [35]:
for key,val  in train_dict1.items():
    print (key+" "+str(len(val)))

7 306
3 1217
5 1038
1 2426
12 395
6 201
0 341
8 763
14 894
9 265
10 274
4 202
2 302
13 147
11 129


Write the pictures that need to be incremented into train.txt

In [56]:
import random
need_aug = {}
need_aug = []
for key,val  in train_dict.items():
    if len(val) >250:
        continue
    leng = len(val)
    num = 250-leng
    i = 0
    while i < num:
        need_aug.append(val[random.randint(0,leng-1)].split("/")[-1].split(".")[0])
        i+=1
print(len(need_aug)) 
print(need_aug[0])
lines = []
names = []
with open("./data/train.txt","r") as fin:
    lines = fin.readlines()
for line in lines:
    tmp = line.split(",")
    names.append(tmp[0].split(".")[0])
with open("./data/need_aug.txt","w") as f_out:
    for name in need_aug:
        if name in names:
            index = names.index(name)
            f_out.write(lines[index].strip()+"\n")



650
2_48


Increase minority class data according to train.txt

In [20]:
from PIL import Image, ImageDraw
import numpy as np
from matplotlib.colors import rgb_to_hsv, hsv_to_rgb
import os
os.environ['DISPLAY'] = ':0'


"""
data enhancement:Data enhancement is actually to make pictures more diverse. 
                 Data enhancement is a very important means to improve the robustness of target detection algorithms.
                 The image can be made more diverse by changing the brightness, image distortion, etc. The changed image is put into the neural network for training to improve the robustness of the network and reduce the impact of various additional factors on recognition.
"""


def rand(a=0, b=1):  # Generate a random number with a value range of (a, b)
    return np.random.rand() * (b - a) + a


#  get_random_data data augment
def get_random_data(annotation_line, input_shape, random=True, max_boxes=20, jitter=.5, hue=.1, sat=1.5, val=1.5, proc_img=True):

    line = annotation_line.split(",")
    path = os.path.join("/home/xiaoer/chenchen/yolov5/data/images/train", line[0])
    print(path)
    image = Image.open(path)
    image.show()
    iw, ih = image.size   # orinal image size
    h, w = input_shape    # model input image size
    box = np.array([np.array(list(map(int, box.split(' ')))) for box in line[1:]])  # Divide the target frame in the picture of the row
    # Scale the image and distort the length and width
    # The distorted picture size may be larger than 416*416, but it will be corrected to 416*416 when adding gray bars
    new_ar = w/h * rand(1-jitter, 1+jitter)/rand(1-jitter, 1+jitter)   # Represents the distortion ratio of the width and height of the original picture, jitter=0, the ratio of the width and height of the original picture remains unchanged, otherwise the width and height of the picture will be distorted to a certain extent
    scale = rand(.25, 2)   # scale controls the zoom ratio of the original image, rand(.25, 2) means zooming between 0.25 and 2, the image may be enlarged or reduced, and rand(.25, 1) will shrink the original image. Adding gray bars to the edges can train the network's ability to detect small targets. rand(1,2) is a certain magnified image
    if new_ar < 1:
        nh = int(scale * h)
        nw = int(nh * new_ar)
    else:
        nw = int(scale * w)
        nh = int(nw / new_ar)
    image = image.resize((nw, nh), Image.BICUBIC)
    # print(nw,nh) # The width and height of the distorted picture
    # Add a gray bar to the extra part of the image to ensure that the size of the image is w,h = 416,416
    dx = int(rand(0, w - nw))
    dy = int(rand(0, h - nh))
    new_image = Image.new('RGB', (w, h), (128, 128, 128))   # (128, 128, 128) is grey color
    new_image.paste(image, (dx, dy))
    image = new_image

    # Whether to flip the image
    flip = rand() < .5  # There is a 50% probability of a flip
    if flip: image = image.transpose(Image.FLIP_LEFT_RIGHT)  # Flip left and right
    # color space  distortion
    # Color space distortion occurs in a color space such as hsv. The hsv color gamut is controlled by hue H, saturation S, and lightness V. Adjust these three values to adjust the ratio of color gamut distortion.
    hue = rand(-hue, hue)
    sat = rand(1, sat) if rand()<.5 else 1/rand(1, sat)
    val = rand(1, val) if rand()<.5 else 1/rand(1, val)
    x = rgb_to_hsv(np.array(image) / 255.)  # After adjusting the picture from the RGB image to the hsv color space, then the color space is distorted
    x[..., 0] += hue
    x[..., 0][x[..., 0] > 1] -= 1
    x[..., 0][x[..., 0] < 0] += 1
    x[..., 1] *= sat
    x[..., 2] *= val
    x[x > 1] = 1
    x[x < 0] = 0
    image_data = hsv_to_rgb(x)  # numpy array, 0 to 1
    # Adjust the box
    # After distorting the original picture, the boxes in the original picture should also be adjusted accordingly
    max_boxes = min(max_boxes, len(box) if len(box) else 0)
    box_data = np.zeros((max_boxes, 5))
    if len(box) > 0:
        np.random.shuffle(box)
        # distrotion adjust
        box[:, [1, 3]] = box[:, [1, 3]] * nw / iw + dx
        box[:, [2, 4]] = box[:, [2, 4]] * nh / ih + dy
        # rotation adjust
        if flip: box[:, [1, 3]] = w - box[:, [3, 1]]

     
        box[:, 1:3][box[:, 1:3] < 0] = 0
        box[:, 3][box[:, 3] > w] = w
        box[:, 4][box[:, 4] > h] = h
        box_w = box[:, 3] - box[:, 1]
        box_h = box[:, 4] - box[:, 2]
        tmp = np.logical_and(box_w > 1, box_h > 1)
        box = box[np.logical_and(box_w > 1, box_h > 1)]  # discard invalid box
        if len(box) > max_boxes: box = box[:max_boxes]
        box_data[:len(box)] = box

    return image_data, box_data


# 原图片绘制展示
def normal_(annotation_line, input_shape):
    """
    random preprocessing for real-time data augmentation
    :param annotation_line: Data enhancement is performed on the image corresponding to row a of the selected data set
    :param input_shape:     input size
    :return:
    """
    line = annotation_line.split(",")  # Split by spaces
    # Get the picture corresponding to the row
    path = os.path.join("/home/xiaoer/chenchen/zengqiang/testquwu/images/train", line[0])
    print(path)
    image = Image.open(path)
    # Get every target box on the picture
    box = np.array([np.array(list(map(int, box.split(' ')[1:]))) for box in line[1:]])
    return image, box

with open("/home/xiaoer/chenchen/yolov5/data/need_aug.txt","r") as f:
    lines = f.readlines()
    """ image_data, box_data = normal_("2_146",line[-1], [640, 640])
    img = image_data


    thickness = 3
    for j in range(len(box_data)):
        left, top, right, bottom = box_data[j][0:]
        draw = ImageDraw.Draw(img)
        for i in range(thickness):
            draw.rectangle([left + i, top + i, right - i, bottom - i], outline=(255, 255, 255))
    img.show()
    img.save("image_save_org.jpg") """
    for line in lines:
        # display enhanced picture
        tmp = line.split(",")
        image_data, box_data = get_random_data(line, [640, 640])
        print(box_data)
        img = Image.fromarray((image_data * 255).astype(np.uint8))
        img.save("/home/xiaoer/chenchen/yolov5/data/images/train/"+tmp[0].split(".")[0]+"_aug.jpg")
        with open("/home/xiaoer/chenchen/yolov5/data/labels/train/"+tmp[0].split(".")[0]+"_aug.txt","w") as f:
            for j in range(len(box_data)):
                thickness = 3
                left, top, right, bottom = box_data[j][1:]
                [img_width, img_height] = img.size
         
                xmin = min(max(0, left), img_width)
                ymin = min(max(0, top), img_height)
                xmax = min(max(0, right), img_width)
                ymax = min(max(0, bottom), img_height)
                if xmax <= xmin or ymax <= ymin:
                    print("Warning: in '{}' xml, there are some bbox w/h <=0")
                    continue
        
                xcenter = xmin + (xmax - xmin) / 2
                ycenter = ymin + (ymax - ymin) / 2
                w = xmax - xmin
                h = ymax - ymin

             
                xcenter = round(xcenter / img_width, 6)
                ycenter = round(ycenter / img_height, 6)
                w = round(w / img_width, 6)
                h = round(h / img_height, 6)
                class_index = int(box_data[j][0])
                info = [str(i) for i in [class_index, xcenter, ycenter, w, h]]
                if j == 0:
                    f.write(" ".join(info))
                else:
                    f.write("\n" + " ".join(info))
           
                draw = ImageDraw.Draw(img)

                for i in range(thickness):
                    draw.rectangle([left + i, top + i, right - i, bottom - i], outline=(255, 255, 255))
        #img.show()
        # save image
        #img.save("image_save_aug.jpg")

delete some data


In [8]:
import os 

train_dict2 = {}
for root, dirs, files in os.walk("/home/xiaoer/chenchen/yolov5/data/labels/train"):

    for f in files:
        with open (os.path.join(root, f)) as fin:
            lines = fin.readlines()
            flag= False
            tep = []
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                temp = line.split(" ")
                tep.append(temp[0])
            if "1" in tep:
                train_dict2[f] =  tep
print(len(train_dict2))

2447


In [32]:
total = 0
with open("data_need_del.txt","w") as f:
    for key,val in  train_dict2.items():
        count = 0
        for v in val:
            if v=="1" and len(val) ==1:
                total+=1
                f.write(key+"\n")
                if os.path.exists("/home/xiaoer/chenchen/yolov5/data/images/train/"+key.split(".")[0]+".jpg"): 
                    print("/home/xiaoer/chenchen/yolov5/data/images/train/"+key.split(".")[0]+".jpg")
                    os.remove("/home/xiaoer/chenchen/yolov5/data/images/train/"+key.split(".")[0]+".jpg") 
                print("/home/xiaoer/chenchen/yolov5/data/labels/train/"+key)
                if os.path.exists("/home/xiaoer/chenchen/yolov5/data/labels/train/"+key): 
                   
                    os.remove("/home/xiaoer/chenchen/yolov5/data/labels/train/"+key)
print(total)

me/xiaoer/chenchen/yolov5/data/labels/train/15_41.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/13_262.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/18_251.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/11_168.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/20_536.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/16_15.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/15_97.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/8_230.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/8_154.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/8_11.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/16_692.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/16_157.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/13_209.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/15_142.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/6_97.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/14_210.txt
/home/xiaoer/chenchen/yolov5/data/labels/train/20_552.txt
/home/xiaoer/chenchen/yolo

In [13]:
print(total)

4


In [36]:
import os 
# 遍历文件夹
count1 = 0
for root, dirs, files in os.walk("/home/xiaoer/chenchen/yolov5/data/labels/train"):
    # 遍历文件
    for f in files:
        count1+=1
count2 = 0
for root, dirs, files in os.walk("/home/xiaoer/chenchen/yolov5/data/images/train"):
     # 遍历文件
    for f in files:
        count2+=1
    
print(count1)
print(count2)

4756
4756


In [1]:
import os 
# 遍历文件夹
train_dict3 = {}
for root, dirs, files in os.walk("/home/xiaoer/chenchen/yolov5/data/labels/val"):
    # 遍历文件
    for f in files:
        with open (os.path.join(root, f)) as fin:
            lines = fin.readlines()
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                temp = line.split(" ")
                if temp[0] not in train_dict3:
                    train_dict3[temp[0]] = []
                train_dict3[temp[0]].append(os.path.join(root, f))
print(len(train_dict3))

11


In [2]:
for key,val  in train_dict3.items():
    print (key+" "+str(len(val)))

3 572
14 510
11 59
0 418
13 529
8 208
6 35
12 188
2 24
10 42
7 10
